In [1]:
# import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# grab our data from the raw github page
url = 'https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv'
df = pd.read_csv(url)
df.shape

(25989, 60)

In [3]:
# We have some useless data in this dataframe (iso_codes like 'International Transport', 'North America (excl. USA)', etc.)
# These are being dropped since they are legitimately useless. We will also drop the 'OWID_WRL' iso_code which is the CO2
# emissions for the globe since it's not necessary for our analysis.

df = df[df['iso_code'].notna()]
df = df[df['iso_code'] != 'OWID_WRL']

# To get an idea for which countries we should forecast, lets sort the countries in descending order with highest emission
# totals for the year 2020 since it's the most recent

df[df['year']==2020].sort_values(['co2'], ascending=False)[:10]

,iso_code,country,year,co2,co2_per_capita,trade_co2,cement_co2,cement_co2_per_capita,coal_co2,coal_co2_per_capita,...,ghg_excluding_lucf_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
4723,CHN,China,2020,10667.887,7.412,NaN,858.233,0.596,7421.101,5.156,...,NaN,NaN,NaN,NaN,NaN,1.439324e+09,NaN,40404.443,28071.824,NaN
24661,USA,United States,2020,4712.771,14.238,NaN,40.795,0.123,888.649,2.685,...,NaN,NaN,NaN,NaN,NaN,3.310026e+08,NaN,24387.388,73677.319,NaN
11277,IND,India,2020,2441.792,1.769,NaN,122.908,0.089,1587.552,1.150,...,NaN,NaN,NaN,NaN,NaN,1.380004e+09,NaN,8884.380,6437.936,NaN
19597,RUS,Russia,2020,1577.136,10.807,NaN,20.306,0.139,356.946,2.446,...,NaN,NaN,NaN,NaN,NaN,1.459345e+08,NaN,7865.165,53895.194,NaN
12336,JPN,Japan,2020,1030.775,8.150,NaN,25.376,0.201,402.979,3.186,...,NaN,NaN,NaN,NaN,NaN,1.264765e+08,NaN,4730.588,37402.909,NaN
11595,IRN,Iran,2020,745.035,8.870,NaN,23.877,0.284,5.114,0.061,...,NaN,NaN,NaN,NaN,NaN,8.399295e+07,NaN,3341.689,39785.350,NaN
9619,DEU,Germany,2020,644.310,7.690,NaN,13.287,0.159,199.077,2.376,...,NaN,NaN,NaN,NaN,NaN,8.378394e+07,NaN,3364.175,40152.984,NaN
20269,SAU,Saudi Arabia,2020,625.508,17.967,NaN,25.593,0.735,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.481387e+07,NaN,2933.478,84261.772,NaN
21590,KOR,South Korea,2020,597.605,11.656,NaN,22.871,0.446,282.883,5.518,...,NaN,NaN,NaN,NaN,NaN,5.126918e+07,NaN,3274.316,63865.188,NaN
11409,IDN,Indonesia,2020,589.500,2.155,NaN,33.804,0.124,300.518,1.099,...,NaN,NaN,NaN,NaN,NaN,2.735236e+08,NaN,2120.519,7752.600,NaN


In [4]:
df.shape

(21689, 60)

In [5]:
# There are a lot of columns that are of little to no interest to us that contain a significant amount of null values
# With the following code we are dropping any column that has more than 1180 null values (the amount of null values in the
# column co2_growth_abs)
for column in df.columns:
    if df[column].isnull().sum() > 1180:
        df = df.drop(column, axis=1)

In [6]:
df.shape

(21689, 11)

In [7]:
# For our analysis we won't need countries that have null values for their CO2 emissions (since this is the whole point
# of our analysis in the first place)
df[df['co2'].isna()]

,iso_code,country,year,co2,co2_per_capita,co2_growth_prct,co2_growth_abs,cumulative_co2,share_global_co2,share_global_cumulative_co2,population
556,ATA,Antarctica,2008,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
557,ATA,Antarctica,2009,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
558,ATA,Antarctica,2010,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
559,ATA,Antarctica,2011,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
560,ATA,Antarctica,2012,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
25195,VEN,Venezuela,1907,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2919560.0
25197,VEN,Venezuela,1909,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2982568.0
25198,VEN,Venezuela,1910,NaN,NaN,0.0,NaN,NaN,NaN,NaN,3011475.0
25199,VEN,Venezuela,1911,NaN,NaN,0.0,NaN,NaN,NaN,NaN,3038601.0


In [8]:
# Keeping only the rows where df.co2 is NOT a NaN value
df = df[df['co2'].notna()]
df.shape

(20841, 11)

In [9]:
# Check to see that our dataframe contains all the relevant information for our analysis
for column in df.columns:
    print(f"Column {column} has {df[column].isnull().sum()} null values")

Column iso_code has 0 null values
Column country has 0 null values
Column year has 0 null values
Column co2 has 0 null values
Column co2_per_capita has 205 null values
Column co2_growth_prct has 218 null values
Column co2_growth_abs has 332 null values
Column cumulative_co2 has 0 null values
Column share_global_co2 has 0 null values
Column share_global_cumulative_co2 has 0 null values
Column population has 205 null values


In [10]:
df.to_csv('CO2_emissions.csv',index=False)

In [11]:
# Check to make sure the stored dataframe is the same size as the one we exported (simply a quality check to make sure it
# exported correctly and that we are in the correct folder)

df_check = pd.read_csv('CO2_emissions.csv')
df_check.shape

(20841, 11)

In [25]:
# Reading in geoJSON data 
import requests
r = requests.get('https://raw.githubusercontent.com/ahalota/Leaflet.CountrySelect/master/countries.geo.json')
geo_data = r.json()

In [32]:
# Export our geoJSON data to it's own JSON file to be read in later
import json
with open('geo_data.json', 'w') as f:
    json.dump(geo_data, f)